In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import Chrome as wdChrome, Firefox as wdFirefox, Edge as wdEdge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.firefox.service import Service as FirefoxService

from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from webdriver_manager.microsoft import EdgeChromiumDriverManager

In [7]:
home_url = 'https://www.google.com/maps/place/Lahore,+Punjab,+Pakistan/@31.4831569,74.1943055,11z/data=!3m1!4b1!4m5!3m4!1s0x39190483e58107d9:0xc23abe6ccc7e2462!8m2!3d31.5203696!4d74.3587473'
driver_type = "chrome"
service = ChromeService(executable_path = ChromeDriverManager().install())
driver = wdChrome(service= service)
driver.get(home_url)

In [37]:
MAP_SEARCH_XPATH = '//*[@id="searchboxinput"]'
LOCATIONS_XPATH = '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div[1]/div[5]'



In [30]:
def search_map(query: str) -> None:
    map_searchbar = driver.find_element(By.XPATH, MAP_SEARCH_XPATH)
    map_searchbar.clear()
    map_searchbar.send_keys(query)
    map_searchbar.send_keys(Keys.RETURN)

In [31]:
search_map("Businesses")

In [32]:
search_map("Cafes")

In [33]:
search_map("resteraunts")

In [35]:
search_map("hotels")

In [42]:
places = driver.find_elements(By.XPATH, LOCATIONS_XPATH)
places


[<selenium.webdriver.remote.webelement.WebElement (session="2db86db38e16ebdf33b207182e9f66d0", element="46814ccb-1a94-48e7-ba5d-83db7e3cdc57")>]